## Food & Health
- 랭크 넘버 대신 오특 이라고 써진 삽입 제품 처리 -> 직전 rank 값 저장 후 rank 데이터 없는 상품일때 직전 rank 값 +1 , 오특이라는 값을 flag 리스트에 추가

### Food

In [113]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

browser = webdriver.Chrome()

url = "https://www.oliveyoung.co.kr/store/main/getBestList.do?dispCatNo=900000100100001&fltDispCatNo=10000020002&pageIdx=1&rowsPerPage=100&t_page=%EB%9E%AD%ED%82%B9&t_click=%ED%8C%90%EB%A7%A4%EB%9E%AD%ED%82%B9_%ED%91%B8%EB%93%9C"
browser.get(url)
sleep(5)

soup = BeautifulSoup(browser.page_source, "html.parser")
itemList = soup.select('.TabsConts.on > ul > li')

dataSave = []
rank_num = 0  # 초기 rank 값

for i, item in enumerate(itemList):
    print(f"\n▶ {i+1}번째 아이템 처리 중...")

    # ▶ rank
    rank_tag = item.select('span.thumb_flag.best')
    if rank_tag:
        rank = int(rank_tag[0].text.strip())
        is_special = False
    else:
        rank = rank_num + 1
        is_special = True
    rank_num = rank

    brandName = item.select('span.tx_brand')[0].text.strip()
    isPb = 1 if brandName == '딜라이트 프로젝트' else 0
    goodsName = item.select('p.tx_name')[0].text.strip()
    salePrice = item.select('p.prd_price > span.tx_cur > span.tx_num')[0].text.strip()

    org_tag = item.select('p.prd_price > span.tx_org > span.tx_num')
    originalPrice = org_tag[0].text.strip() if org_tag else salePrice

    flag_tags = item.select('p.prd_flag > span.icon_flag')
    flagList = [tag.text.strip() for tag in flag_tags] if flag_tags else []
    if is_special:
        flagList.append('오특')

    data = {
        'rank': rank,
        'brandName': brandName,
        'isPb': isPb,
        'goodsName': goodsName,
        'salePrice': salePrice,
        'originalPrice': originalPrice,
        'flagList': flagList
    }

    link_tag = item.select_one('a.prd_thumb')
    goodsUrl = link_tag['href'] if link_tag and 'href' in link_tag.attrs else ''

    if goodsUrl:
        browser.get(goodsUrl)
        sleep(3)

        try:
            review_btn = browser.find_element("css selector", 'li#reviewInfo a.goods_reputation')
            review_btn.click()
            sleep(2)

            detail_soup = BeautifulSoup(browser.page_source, "html.parser")

            # ▶ 필수 리뷰 정보 (예외처리 없이 바로 추출)
            totalComment = detail_soup.select_one('p.img_face > em').text.strip()
            numOfReviews = detail_soup.select_one('p.total > em').text.strip()

            li_tags = detail_soup.select('ul.graph_list > li > span.per')
            pctOf5 = int(li_tags[0].get_text(strip=True).replace('%', ''))
            pctOf4 = int(li_tags[1].get_text(strip=True).replace('%', ''))
            pctOf3 = int(li_tags[2].get_text(strip=True).replace('%', ''))
            pctOf2 = int(li_tags[3].get_text(strip=True).replace('%', ''))
            pctOf1 = int(li_tags[4].get_text(strip=True).replace('%', ''))

            # ▶ 선택 리뷰 영역 (없을 수도 있음)
            packing_data = {'pakingGood': 0, 'pakingMiddle': 0, 'pakingBad': 0}
            exp_data = {'expLong': 0, 'expMiddle': 0, 'expShort': 0}
            taste_data = {'tasteGood': 0, 'tasteMiddle': 0, 'tasteBad': 0}

            review_list = detail_soup.select('div.poll_all.clrfix > .poll_type2.type3')
            if len(review_list) >= 3:
                categories = [
                    ('paking', ['Good', 'Middle', 'Bad']),
                    ('exp', ['Long', 'Middle', 'Short']),
                    ('taste', ['Good', 'Middle', 'Bad'])
                ]

                for idx, (prefix, suffixes) in enumerate(categories):
                    item_list = review_list[idx].select('dd li')
                    for i, suffix in enumerate(suffixes):
                        key = f"{prefix}{suffix}"
                        try:
                            value = int(item_list[i].select_one('em').get_text(strip=True).replace('%', ''))
                        except:
                            value = 0
                        if prefix == 'paking':
                            packing_data[key] = value
                        elif prefix == 'exp':
                            exp_data[key] = value
                        elif prefix == 'taste':
                            taste_data[key] = value
            else:
                print("→ 상세 리뷰 항목 없음 (packing/exp/taste)")

            # ▶ 상세정보 저장
            data.update({
                'totalComment': totalComment,
                'numOfReviews': numOfReviews,
                'pctOf5': pctOf5,
                'pctOf4': pctOf4,
                'pctOf3': pctOf3,
                'pctOf2': pctOf2,
                'pctOf1': pctOf1,
                **packing_data,
                **exp_data,
                **taste_data
            })

            print(f"✓ 리뷰 데이터 수집 완료: {goodsName}")

        except Exception as e:
            print(f"[오류] 리뷰 탭 클릭 또는 파싱 실패: {e}")
            # pctOf 계열은 무조건 있다고 가정하므로, 에러 시엔 빈값만 설정
            data.update({
                'pakingGood': 0, 'pakingMiddle': 0, 'pakingBad': 0,
                'expLong': 0, 'expMiddle': 0, 'expShort': 0,
                'tasteGood': 0, 'tasteMiddle': 0, 'tasteBad': 0
            })

    dataSave.append(data)

# ▶ DataFrame 생성
raw_food = pd.DataFrame(dataSave)

# ▶ 브라우저 종료
browser.quit()



▶ 1번째 아이템 처리 중...
✓ 리뷰 데이터 수집 완료: 프로티원 단백질쉐이크 파우치형 40g 5종 산리오

▶ 2번째 아이템 처리 중...
✓ 리뷰 데이터 수집 완료: [7월 올영픽/띠부씰 포함] 딜라이트 프로젝트 베이글칩 6종 택1 (산리오 에디션)

▶ 3번째 아이템 처리 중...
✓ 리뷰 데이터 수집 완료: [7월 올영픽/1+1] 딜라이트 프로젝트 단백질쉐이크 4종 택1 (산리오 에디션)

▶ 4번째 아이템 처리 중...
✓ 리뷰 데이터 수집 완료: [7월 올영픽] 딜라이트 프로젝트 베이글칩&단백질쉐이크 (산리오 에디션)

▶ 5번째 아이템 처리 중...
✓ 리뷰 데이터 수집 완료: [7월 올영픽] 플라이밀 단백질쉐이크 45g 9종 택1

▶ 6번째 아이템 처리 중...
✓ 리뷰 데이터 수집 완료: 딜라이트 프로젝트 단백질쉐이크 단품/7개입 8종 택1

▶ 7번째 아이템 처리 중...
✓ 리뷰 데이터 수집 완료: [7월 파워팩 단독] 랩노쉬 단백쿠키 40g 더블초코 더블 기획 외 4종

▶ 8번째 아이템 처리 중...
✓ 리뷰 데이터 수집 완료: [7월 올영픽] 플라이밀 단백질쉐이크 피넛버터 45g

▶ 9번째 아이템 처리 중...
✓ 리뷰 데이터 수집 완료: [7월 올영픽] 플라이밀 단백질쉐이크 쿠키앤크림 45g

▶ 10번째 아이템 처리 중...
✓ 리뷰 데이터 수집 완료: [단독기획/1+1] 플라이밀 단백질쉐이크 쿠키앤크림, 피넛버터 45g 2종

▶ 11번째 아이템 처리 중...
✓ 리뷰 데이터 수집 완료: 딜라이트 프로젝트 베이글칩 7종 택1

▶ 12번째 아이템 처리 중...
✓ 리뷰 데이터 수집 완료: [7월 올영픽] 딜라이트 프로젝트 명인 찹쌀오징어채부각 50g 2종 택1 (오리지널/청양마요)

▶ 13번째 아이템 처리 중...
✓ 리뷰 데이터 수집 완료: [7월올영픽/1+1] 빼바 소프트 크런치 프로틴바 2종 산리오캐릭터즈 태닝 헬로키티

▶ 14번째 아이템 처리 중...
✓ 리뷰 데이터 수집 완료: [7/1 하루특가] 라이블링 클

In [114]:
raw_food

,rank,brandName,isPb,goodsName,salePrice,originalPrice,flagList,totalComment,numOfReviews,pctOf5,...,pctOf1,pakingGood,pakingMiddle,pakingBad,expLong,expMiddle,expShort,tasteGood,tasteMiddle,tasteBad
0,1,프로티원,0,프로티원 단백질쉐이크 파우치형 40g 5종 산리오,"3,900","3,900","[1+1, 오늘드림]",최고,"29,223",87,...,1,72,28,0,72,28,0,66,34,0
1,2,딜라이트 프로젝트,1,[7월 올영픽/띠부씰 포함] 딜라이트 프로젝트 베이글칩 6종 택1 (산리오 에디션),"2,430","2,700","[2+1, 쿠폰, 오늘드림]",최고,138,88,...,0,74,26,0,70,30,0,75,25,0
2,3,딜라이트 프로젝트,1,[7월 올영픽/1+1] 딜라이트 프로젝트 단백질쉐이크 4종 택1 (산리오 에디션),"3,900","3,900","[1+1, 오늘드림]",최고,39,100,...,0,74,26,0,64,36,0,82,18,0
3,4,딜라이트 프로젝트,1,[7월 올영픽] 딜라이트 프로젝트 베이글칩&단백질쉐이크 (산리오 에디션),"2,430","2,700","[2+1, 쿠폰, 오늘드림]",최고,25,92,...,0,80,20,0,64,36,0,76,24,0
4,5,플라이밀,0,[7월 올영픽] 플라이밀 단백질쉐이크 45g 9종 택1,"3,900","3,900","[1+1, 오늘드림]",최고,"39,368",85,...,1,70,30,0,63,37,1,68,30,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,온브릭스,0,"[온브릭스] 신선함을 담은, 항공직송 워싱턴 생체리","18,960","23,700","[쿠폰, 무배]",최고,31,97,...,0,0,0,0,0,0,0,0,0,0
96,97,매일유업,0,[매일유업] 앱솔루트 유기농 궁 분유 1단계(0~6개월) 800g 3캔,"105,000","105,000",[],최고,26,100,...,0,96,4,0,88,12,0,88,12,0
97,98,오설록,0,오설록 말차 프레첼 대용량 (125g),"7,350","10,500","[세일, 오늘드림]",최고,"1,166",77,...,2,74,26,0,64,36,0,63,28,9
98,99,랩노쉬,0,랩노쉬 저당 웨하스 53g 2종(초코바닐라/초코딸기) 택1,"3,500","3,500","[2+1, 오늘드림]",최고,"5,704",89,...,0,71,29,1,68,32,0,76,24,1


In [119]:
raw_food.to_csv('20250701_raw_food.csv', index=False)

In [29]:
driver.quit()

In [120]:
dd = pd.read_csv('20250701_raw_food.csv')
dd

,rank,brandName,isPb,goodsName,salePrice,originalPrice,flagList,totalComment,numOfReviews,pctOf5,...,pctOf1,pakingGood,pakingMiddle,pakingBad,expLong,expMiddle,expShort,tasteGood,tasteMiddle,tasteBad
0,1,프로티원,0,프로티원 단백질쉐이크 파우치형 40g 5종 산리오,"3,900","3,900","['1+1', '오늘드림']",최고,"29,223",87,...,1,72,28,0,72,28,0,66,34,0
1,2,딜라이트 프로젝트,1,[7월 올영픽/띠부씰 포함] 딜라이트 프로젝트 베이글칩 6종 택1 (산리오 에디션),"2,430","2,700","['2+1', '쿠폰', '오늘드림']",최고,138,88,...,0,74,26,0,70,30,0,75,25,0
2,3,딜라이트 프로젝트,1,[7월 올영픽/1+1] 딜라이트 프로젝트 단백질쉐이크 4종 택1 (산리오 에디션),"3,900","3,900","['1+1', '오늘드림']",최고,39,100,...,0,74,26,0,64,36,0,82,18,0
3,4,딜라이트 프로젝트,1,[7월 올영픽] 딜라이트 프로젝트 베이글칩&단백질쉐이크 (산리오 에디션),"2,430","2,700","['2+1', '쿠폰', '오늘드림']",최고,25,92,...,0,80,20,0,64,36,0,76,24,0
4,5,플라이밀,0,[7월 올영픽] 플라이밀 단백질쉐이크 45g 9종 택1,"3,900","3,900","['1+1', '오늘드림']",최고,"39,368",85,...,1,70,30,0,63,37,1,68,30,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,온브릭스,0,"[온브릭스] 신선함을 담은, 항공직송 워싱턴 생체리","18,960","23,700","['쿠폰', '무배']",최고,31,97,...,0,0,0,0,0,0,0,0,0,0
96,97,매일유업,0,[매일유업] 앱솔루트 유기농 궁 분유 1단계(0~6개월) 800g 3캔,"105,000","105,000",[],최고,26,100,...,0,96,4,0,88,12,0,88,12,0
97,98,오설록,0,오설록 말차 프레첼 대용량 (125g),"7,350","10,500","['세일', '오늘드림']",최고,"1,166",77,...,2,74,26,0,64,36,0,63,28,9
98,99,랩노쉬,0,랩노쉬 저당 웨하스 53g 2종(초코바닐라/초코딸기) 택1,"3,500","3,500","['2+1', '오늘드림']",최고,"5,704",89,...,0,71,29,1,68,32,0,76,24,1
